In [5]:
from pathlib import Path
import shutil

import gymnasium as gym
import torch

from deep_q import DeepQFunc, DeepQFuncTrainer, DeepQFuncTester, ReplayBuffer, Discrete1ContinuousAction
from env import Env


#### 使用CarPole-V1 环境，测试简单的Deep Q 如何处理连续的State空间和离散的Action空间

In [6]:
GYM_ENV_NAME = 'CartPole-v1'
_train_gym_env = gym.make(GYM_ENV_NAME)

# 打印查看环境的动作空间和状态空间 
action_nums, state_space = _train_gym_env.action_space.n, _train_gym_env.observation_space
print(f'action num: {action_nums}, space: {state_space}')

TRAIN_EPOCH = 300
HIDDEN_DIM = 256
LEARNING_RATE = 2e-3
GAMMA = 0.99

# 使用指数递减的epsilon-greedy策略
START_EPSILON = 0.5
END_EPSILON = 0.05
DECAY_RATE = 0.99
EPSILON_LIST = [max(START_EPSILON * (DECAY_RATE ** i), END_EPSILON) for i in range(TRAIN_EPOCH)]


log_path = Path('./logs/run2')
if log_path.exists():
    shutil.rmtree(log_path)

# _USE_CUDA = True and torch.cuda.is_available()
_USE_CUDA = False and torch.cuda.is_available()

q_func = DeepQFunc(state_space.shape[0], 
                   action_nums, 
                   hidden_dim=HIDDEN_DIM, 
                   device=torch.device('cuda') if _USE_CUDA else None)

env = Env(_train_gym_env)

replay_buffer = ReplayBuffer(10000)
q_func_trainer = DeepQFuncTrainer(q_func=q_func, 
                                  env=env,
                                  replay_buffer=replay_buffer,
                                  learning_rate=LEARNING_RATE,
                                  batch_size=64,
                                  gamma=GAMMA,
                                  epsilon_list=EPSILON_LIST,
                                  logger_folder=log_path)


action num: 2, space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [7]:
q_func_trainer.train(train_epoch=TRAIN_EPOCH, 
                     max_steps=1000, 
                     minimal_replay_size_to_train=64 * 10,
                     target_q_update_freq=10)

 26%|██▋       | 79/300 [00:00<00:02, 84.39it/s, reward=21, step=21] 


KeyboardInterrupt: 

#### 开始测试

In [ ]:
_render_env = Env(gym.make(GYM_ENV_NAME, render_mode='human'))
q_func_tester = DeepQFuncTester(
    q_func=q_func.to('cpu'),
    env=_render_env
)

In [ ]:
q_func_tester.test(2000)

In [ ]:
p = Path('./model')
if not p.exists():
    p.mkdir()
q_func.save(Path('./model/trained_for_cartpole.pth'))

In [ ]:
q_func_from_checkpoint = DeepQFunc(state_space.shape[0], 
                   action_nums, 
                   hidden_dim=HIDDEN_DIM)
q_func_from_checkpoint.load(Path('./model/trained_for_cartpole.pth'))

In [ ]:
q_func_tester = DeepQFuncTester(
    q_func=q_func_from_checkpoint.to('cpu'),
    env=_render_env
)

In [ ]:
q_func_tester.test(2000)

#### 使用Pendulum-v1 测试Double Q Learning对Q值系统高估的处理能力

In [ ]:
GYM_ENV_NAME = 'Pendulum-v1'
_train_gym_env = gym.make(GYM_ENV_NAME)

# 打印查看环境的动作空间和状态空间 
action_space, state_space = _train_gym_env.action_space, _train_gym_env.observation_space
print(f'action: {action_space}, space: {state_space}')

BINS = 11


TRAIN_EPOCH = 1000
HIDDEN_DIM = 256
LEARNING_RATE = 2e-3
GAMMA = 0.99

# 使用指数递减的epsilon-greedy策略
START_EPSILON = 0.5
END_EPSILON = 0.05
DECAY_RATE = 0.99
EPSILON_LIST = [max(START_EPSILON * (DECAY_RATE ** i), END_EPSILON) for i in range(TRAIN_EPOCH)]


log_path = Path('./logs/pendulum/run_dqn')
if log_path.exists():
    shutil.rmtree(log_path)

# _USE_CUDA = True and torch.cuda.is_available()
_USE_CUDA = False and torch.cuda.is_available()

q_func = DeepQFunc(state_space.shape[0], 
                   BINS, 
                   hidden_dim=HIDDEN_DIM, 
                   device=torch.device('cuda') if _USE_CUDA else None)

env = Env(_train_gym_env)

replay_buffer = ReplayBuffer(10000)
q_func_trainer = DeepQFuncTrainer(q_func=q_func, 
                                  env=env,
                                  replay_buffer=replay_buffer,
                                  learning_rate=LEARNING_RATE,
                                  batch_size=64,
                                  gamma=GAMMA,
                                  epsilon_list=EPSILON_LIST,
                                  logger_folder=log_path,
                                  action_converter=Discrete1ContinuousAction(action_space.low, action_space.high, BINS))

action: Box(-2.0, 2.0, (1,), float32), space: Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)


In [ ]:
q_func_trainer.train(train_epoch=TRAIN_EPOCH, 
                     max_steps=1000, 
                     minimal_replay_size_to_train=64 * 10,
                     target_q_update_freq=10)

In [ ]:
_render_env = Env(gym.make(GYM_ENV_NAME, render_mode='human'))
q_func_tester = DeepQFuncTester(
    q_func=q_func.to('cpu'),
    env=_render_env,
    action_converter=Discrete1ContinuousAction(action_space.low, action_space.high, BINS)
)

In [ ]:
q_func_tester.test(1000)

#### 使用Double DQN 进行训练

In [ ]:
from deep_q import DoubleQFuncTrainer

log_path = Path('./logs/pendulum/run_double_dqn3')
if log_path.exists():
    shutil.rmtree(log_path)

q_func2 = DeepQFunc(state_space.shape[0], 
                   BINS, 
                   hidden_dim=HIDDEN_DIM, 
                   device=torch.device('cuda') if _USE_CUDA else None)

replay_buffer = ReplayBuffer(10000)
q_func_trainer = DoubleQFuncTrainer(q_func=q_func2, 
                                  env=env,
                                  replay_buffer=replay_buffer,
                                  learning_rate=LEARNING_RATE,
                                  batch_size=64,
                                  gamma=GAMMA,
                                  epsilon_list=EPSILON_LIST,
                                  logger_folder=log_path,
                                  action_converter=Discrete1ContinuousAction(action_space.low, action_space.high, BINS))

In [ ]:
q_func_trainer.train(train_epoch=TRAIN_EPOCH, 
                     max_steps=1000, 
                     minimal_replay_size_to_train=64 * 10,
                     target_q_update_freq=10)

  0%|          | 0/1000 [00:00<?, ?it/s]f:\conda\envs\quant\Lib\site-packages\torch\utils\_device.py:78: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  return func(*args, **kwargs)
  0%|          | 2/1000 [00:00<00:53, 18.82it/s, reward=-1.83e+3, step=200]f:\ws\rf_learning\deep_q.py:295: UserWarning: Using a target size (torch.Size([64, 1, 64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = torch.nn.functional.mse_loss(q_values_now_value, target_q_values.unsqueeze(1))


q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1])

  0%|          | 4/1000 [00:00<02:15,  7.36it/s, reward=-1.26e+3, step=200]

q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1])

  0%|          | 4/1000 [00:00<02:15,  7.36it/s, reward=-1.63e+3, step=200]

...

  2%|▎         | 25/1000 [00:08<06:32,  2.48it/s, reward=-902, step=200]

q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1])

  2%|▎         | 25/1000 [00:09<05:54,  2.75it/s, reward=-902, step=200]

q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1]), target_q_values dim: torch.Size([64, 64])
q_values_now_value dim: torch.Size([64, 1])

KeyboardInterrupt: 

In [ ]:
_render_env = Env(gym.make(GYM_ENV_NAME, render_mode='human'))
q_func_tester = DeepQFuncTester(
    q_func=q_func2.to('cpu'),
    env=_render_env,
    action_converter=Discrete1ContinuousAction(action_space.low, action_space.high, BINS)
)

In [ ]:
q_func_tester.test(1000)